## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.6 MB/s 
     |████████████████████████████████| 182 kB 89.7 MB/s 
     |████████████████████████████████| 7.6 MB 74.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 6.8 MB/s 
     |████████████████████████████████| 132 kB 89.8 MB/s 
     |████████████████████████████████| 212 kB 86.8 MB/s 
     |████████████████████████████████| 127 kB 94.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 6.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [2]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

# from InstructABSA.utils import T5ATE
# from InstructABSA.data_prep import ModelReadyData

In [ ]:
rest_train_file_path = './Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'
rest_val_file_path = './Data/semeval14/ABSA_TrainData/Rest_val.csv'
laptops_val_file_path = './Data/semeval14/ABSA_TrainData/Lap_val.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)
res_val_df = pd.read_csv(rest_val_file_path)
lap_val_df = pd.read_csv(laptops_val_file_path)

lap_tr_df =pd.concat([lap_tr_df, lap_val_df]).reset_index(drop = True)
res_tr_df =pd.concat([res_tr_df, res_val_df]).reset_index(drop = True)

In [ ]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()

# Get the input text into the required format
bos_instruction_lap = """Definition: The output will be the aspects (both implicit and explicit) which have an associated opinion that are extracted from the input text. In cases where there are no aspects the output should be noaspectterm.
Positive example 1-
input: I charge it at night and skip taking the cord with me because of the good battery life.
output: battery life
Positive example 2-
input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!.
output: features, iChat, Photobooth, garage band
Negative example 1-
input: Speaking of the browser, it too has problems.
output: browser
Negative example 2-
input: The keyboard is too slick.
output: keyboard
Neutral example 1-
input: I took it back for an Asus and same thing- blue screen which required me to remove the battery to reset.
output: battery
Neutral example 2-
input: Nightly my computer defrags itself and runs a virus scan.
output: virus scan
Now complete the following example-
input: """

bos_instruction_res = """Definition: The output will be the aspects (both implicit and explicit) which have an associated opinion that are extracted from the input text. In cases where there are no aspects the output should be noaspectterm.
Positive example 1-
input: With the great variety on the menu , I eat here often and never get bored.
output: menu
Positive example 2- 
input: Great food, good size menu, great service and an unpretensious setting.
output: food, menu, service, setting
Negative example 1-
input: They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it.
output: toast, mayonnaise, bacon, ingredients, plate
Negative example 2-
input: The seats are uncomfortable if you are sitting against the wall on wooden benches.
output: seats
Neutral example 1-
input: I asked for seltzer with lime, no ice.
output: seltzer with lime
Neutral example 2-
input: They wouldnt even let me finish my glass of wine before offering another.
output: glass of wine
Now complete the following example-
input: """
eos_instruction = ' Output:'
res_tr_df = model_data.create_data_in_ate_format(res_tr_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_res, eos_instruction)
lap_tr_df = model_data.create_data_in_ate_format(lap_tr_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_lap, eos_instruction)
res_te_df = model_data.create_data_in_ate_format(res_te_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_res, eos_instruction)
lap_te_df = model_data.create_data_in_ate_format(lap_te_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_lap, eos_instruction)

In [ ]:
print(res_tr_df.iloc[0]['text'] + ' ' + res_tr_df.iloc[0]['labels'])

Definition: The output will be the aspects (both implicit and explicit) which have an associated opinion that are extracted from the input text. In cases where there are no aspects the output should be noaspectterm.
Positive example-
input: With the great variety on the menu , I eat here often and never get bored.
output: menu
Negative example-
input: They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it.
output: toast, mayonnaise, bacon, ingredients, plate
Neutral example-
input: I asked for seltzer with lime, no ice.
output: seltzer with lime
Now complete the following example-
input: But the staff was so horrible to us. Output: staff


In [ ]:
# Experimentation
experiment_id = 'restaurants'
experiment_name = 'restaurants_instruct_pos_neg_neut'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATE(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATE')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
id_dataset, ood_dataset, id_tokenized_dataset, ood_tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--allenai--tk-instruct-base-def-pos/snapshots/2455b50f8c27ddfe613714466b7a6ffc5f4d338d/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--allenai--tk-instruct-base-def-pos/snapshots/2455b50f8c27ddfe613714466b7a6ffc5f4d338d/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--allenai--tk-instruct-base-def-pos/snapshots/2455b50f8c27ddfe613714466b7a6ffc5f4d338d/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--allenai--tk-instruct-base-def-pos/snapshots/2455b50f8c27ddfe613714466b7a6ffc5f4d338d/tokenizer_config.json


Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--tk-instruct-base-def-pos/snapshots/2455b50f8c27ddfe613714466b7a6ffc5f4d338d/config.json
Model config T5Config {
  "_name_or_path": "allenai/tk-instruct-base-def-pos",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 32100
}

loading weigh

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/checkpoints


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
print(id_dataset['train']['text'][50])
print()
id_dataset['train']['labels'][50]

Definition: The output will be the aspects (both implicit and explicit) which have an associated opinion that are extracted from the input text. In cases where there are no aspects the output should be noaspectterm.
Positive example-
input: With the great variety on the menu , I eat here often and never get bored.
output: menu
Negative example-
input: They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it.
output: toast, mayonnaise, bacon, ingredients, plate
Neutral example-
input: I asked for seltzer with lime, no ice.
output: seltzer with lime
Now complete the following example-
input: The pesto pizza was excellent, thin-crust pizza with a nice amount of spicy Italian cheese that I'd never heard of before. Output:



'pesto pizza, Italian cheese, thin-crust pizza'

In [ ]:
# Train Model
model_trainer = t5_exp.train(id_tokenized_dataset, **training_args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3095
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 776
  Number of trainable parameters = 247534848
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a 

Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.308452
2,No log,0.262039
3,0.508400,0.245964
4,0.508400,0.247826


***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set (Restaurants Domain)
id_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set (Restaurants Domain)
id_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'validation')

# Get prediction labels - Training set (Laptops Domain)
ood_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set (Laptops Domain)
ood_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3095
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/checkpoints


***** Running Prediction *****
  Num examples = 800
  Batch size = 16


The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3085
  Batch size = 16


***** Running Prediction *****
  Num examples = 800
  Batch size = 16


In [ ]:
# Add new column in the respective dataframes
id_tr_df = pd.DataFrame(id_dataset['train'])
id_te_df = pd.DataFrame(id_dataset['validation'])
ood_tr_df = pd.DataFrame(ood_dataset['train'])
ood_te_df = pd.DataFrame(ood_dataset['validation'])

id_tr_df['pred_labels'] =  id_tr_pred_labels
id_te_df['pred_labels'] =  id_te_pred_labels
ood_tr_df['pred_labels'] = ood_tr_pred_labels
ood_te_df['pred_labels'] =  ood_te_pred_labels

# Metrics
print('In domain train accuracy: ', id_tr_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(id_tr_df))
print('In domain test accuracy: ', id_te_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(id_te_df))

ood_df = pd.concat([ood_tr_df, ood_te_df])
print('Out of domain train accuracy: ', ood_tr_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(ood_tr_df))
print('Out of domain test accuracy: ', ood_te_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(ood_te_df))
print('Out of domain accuracy: ', ood_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(ood_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
id_tr_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
id_te_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + id_te_filename), index = False)

ood_tr_filename = t5_exp.get_csv_filename(experiment_id)[2]
ood_tr_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + ood_tr_filename), index = False)

ood_te_filename = t5_exp.get_csv_filename(experiment_id)[3]
ood_te_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + ood_te_filename), index = False)

In domain train accuracy:  0.8639741518578352
In domain test accuracy:  0.81625
Out of domain train accuracy:  0.6596434359805511
Out of domain test accuracy:  0.64625
Out of domain accuracy:  0.6568854568854569


In [5]:
id_trdf = pd.read_csv('./T5/ATE/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/posnegneuttk_restaurants_train.csv')
id_tedf = pd.read_csv('./T5/ATE/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/posnegneuttk_restaurants_test.csv')
ood_tedf = pd.read_csv('./T5/ATE/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/posnegneuttk_laptops_test.csv')

def get_f1_acc(dataframe, true_col, pred_col):
    total_pred = 0
    total_gt = 0
    tp = 0
    for gt, pred in zip(dataframe[true_col][:], dataframe[pred_col][:]):
        gt_list = gt.split(', ')
        pred_list = pred.split(', ')
        total_pred+=len(pred_list)
        total_gt+=len(gt_list)
        for gt_val in gt_list:
            for pred_val in pred_list:
                if pred_val in gt_val:
                    tp+=1
    p = tp/total_pred
    r = tp/total_gt
    return p, r, 2*p*r/(p+r)

print('Train precision:', get_f1_acc(id_trdf, 'labels', 'pred_labels')[0])
print('Test precision:', get_f1_acc(id_tedf, 'labels', 'pred_labels')[0])
print('OOD Test precision:', get_f1_acc(ood_tedf, 'labels', 'pred_labels')[0])

print('Train recall:', get_f1_acc(id_trdf, 'labels', 'pred_labels')[1])
print('Test recall:', get_f1_acc(id_tedf, 'labels', 'pred_labels')[1])
print('OOD Test recall:', get_f1_acc(ood_tedf, 'labels', 'pred_labels')[1])

print('Train F1:', get_f1_acc(id_trdf, 'labels', 'pred_labels')[2])
print('Test F1:', get_f1_acc(id_tedf, 'labels', 'pred_labels')[2])
print('OOD Test F1:', get_f1_acc(ood_tedf, 'labels', 'pred_labels')[2])

Train precision: 0.9482758620689655
Test precision: 0.934984520123839
OOD Test precision: 0.7936694021101993
Train recall: 0.9256181175981716
Test recall: 0.9075882794891059
OOD Test recall: 0.6560077519379846
Train F1: 0.9368100094627274
Test F1: 0.9210827296988182
OOD Test F1: 0.7183023872679045
